In [1]:
import pandas as pd
import numpy as np
import h5py

In [2]:
test = pd.read_hdf('test.h5','data')
train = pd.read_hdf('train.h5','data')

In [4]:
with h5py.File('product_info.h5', 'r') as ipt:
    productinfo = ipt['info'][:]

In [11]:
productindex = np.load('pindex.npy',allow_pickle=True).item()

In [14]:
eventmap = {'cart':5, 'purchase':10, 'remove_from_cart':-5, 'view':1}

In [63]:
productViews = np.sum(productinfo['view'])
productCarts = np.sum(productinfo['cart'])
productRcarts = np.sum(productinfo['remove_from_cart'])
productPurchases = np.sum(productinfo['purchase'])


In [64]:
plen = 100
def transferTrain(df,eventmap):
    gr = df.groupby('user_id').filter(lambda x: len(x)>=10).groupby('user_id')
    xtrain = np.zeros((len(gr),plen,11))
    ytrain = np.zeros(len(gr),dtype=int)
    yreal = np.zeros((len(gr),plen),dtype=int)
    for i,(usr_id,ur) in enumerate(gr):
        predict = ur.iloc[-1]['product_id']
        if np.where(ur.iloc[:-1]['product_id']==predict)[0].shape[0]==0:
            predict = ur.iloc[-2]['product_id']
        for j,(pid, upr) in enumerate(ur.iloc[:-1].groupby('product_id')):
            for et in upr['event_type']:
                if et==eventmap['cart']:
                    xtrain[i,j,0] +=1
                elif et==eventmap['purchase']:
                    xtrain[i,j,1] +=1
                elif et==eventmap['remove_from_cart']:
                    xtrain[i,j,2] +=1
                elif et==eventmap['view']:
                    xtrain[i,j,3] +=1
            xtrain[i,j,4] = productinfo[productindex[pid]]['cart-view']
            xtrain[i,j,5] = productinfo[productindex[pid]]['purchase-view']
            xtrain[i,j,6] = productinfo[productindex[pid]]['purchase-cart']
            xtrain[i,j,7] = productinfo[productindex[pid]]['view']/productViews
            xtrain[i,j,8] = productinfo[productindex[pid]]['cart']/productCarts
            xtrain[i,j,9] = productinfo[productindex[pid]]['remove_from_cart']/productRcarts
            xtrain[i,j,10] = productinfo[productindex[pid]]['purchase']/productPurchases
            if pid == predict:
                ytrain[i] = j
            yreal[i][j] = pid
            if j>=(plen-1):
                break
        yreal[i][j:] = ur.iloc[-2]['product_id']
        #if i>2:
        #    break
    return xtrain,ytrain,yreal
def transferTest(df,eventmap):
    gr = df.groupby('user_id')
    xtrain = np.zeros((len(gr),plen,11))
    yreal = np.zeros((len(gr),plen),dtype=int)
    for i,(usr_id,ur) in enumerate(gr):
        for j,(pid, upr) in enumerate(ur.groupby('product_id')):
            for et in upr['event_type']:
                if et==eventmap['cart']:
                    xtrain[i,j,0] +=1
                elif et==eventmap['purchase']:
                    xtrain[i,j,1] +=1
                elif et==eventmap['remove_from_cart']:
                    xtrain[i,j,2] +=1
                elif et==eventmap['view']:
                    xtrain[i,j,3] +=1
            xtrain[i,j,4] = productinfo[productindex[pid]]['cart-view']
            xtrain[i,j,5] = productinfo[productindex[pid]]['purchase-view']
            xtrain[i,j,6] = productinfo[productindex[pid]]['purchase-cart']
            xtrain[i,j,7] = productinfo[productindex[pid]]['view']/productViews
            xtrain[i,j,8] = productinfo[productindex[pid]]['cart']/productCarts
            xtrain[i,j,9] = productinfo[productindex[pid]]['remove_from_cart']/productRcarts
            xtrain[i,j,10] = productinfo[productindex[pid]]['purchase']/productPurchases
            yreal[i][j] = pid
            if j>=(plen-1):
                break
        yreal[i][j:] = ur.iloc[-1]['product_id']
        #if i>2:
        #    break
    return xtrain,yreal

In [65]:
xtest,ytestreal=transferTest(test,eventmap)

In [66]:
xtrain,ytrain,ytrainreal=transferTrain(train,eventmap)

In [17]:
import torch
from torch import nn
import torch.optim as optim
import torch.utils.data as Data
import torch.nn.functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"

In [67]:
plen=100
nfeature=11
x_train = torch.from_numpy(xtrain).view(-1,plen,nfeature).cuda()
y_train = torch.from_numpy(ytrain).view(-1,1).cuda()
x_test = torch.from_numpy(xtest).view(-1,plen,nfeature).cuda()

In [ ]:
y_train.shape

In [68]:
batch_size=128
seperate=5000
train_dataset = Data.TensorDataset(x_train[:seperate],y_train[:seperate])
train_loader = Data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=False)
test_dataset = Data.TensorDataset(x_train[seperate:],y_train[seperate:])
test_loader = Data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [59]:
class Net(torch.nn.Module):
    def __init__(self, nfeature, out):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(nfeature,nfeature*4)
        self.fc2 = torch.nn.Linear(nfeature*4,nfeature*8)
        self.fc3 = torch.nn.Linear(nfeature*8,nfeature*8)
        self.fc = torch.nn.Linear(nfeature*4,out)
        self.conv1 = torch.nn.Conv1d(1,1,(1,3),padding=(0,1))
        self.fc4 = torch.nn.Linear(nfeature*8,nfeature*4)
    def forward(self, x):
        x = F.relu(self.hidden(x))
        #print(x.shape)
        x = F.relu(self.fc2(x))
        #print(x.shape)
        x = F.relu(self.fc3(x))
        #print(x.shape)
        x = F.relu(self.fc4(x))
        x = self.fc(x)
        #x = self.hidden(x)
        return x
model = Net(11,1).to(device)

In [69]:
learning_rate=0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#criterion = nn.MSELoss()
criterion=nn.CrossEntropyLoss()

In [77]:
epochs = 64
errrate = 0
for epoch in range(epochs):
    model.train()
    for batch,(x,y) in enumerate(train_loader):
        ypred = model(x.float())
        loss = criterion(ypred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    size = len(test_loader.dataset)
    test_loss, correct = 0,0
    with torch.no_grad():
        for x,y in test_loader:
            x = x.float()
            pred = model(x)
            test_loss += criterion(pred,y).item()
            correct += (pred.argmax(1) ==y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    if correct>errrate:
        errrate = correct
        torch.save(model.state_dict(),'itemUser2.pth')
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss:{test_loss:>8f} \n')

Test Error: 
 Accuracy: 28.6%, Avg loss:0.020558 

Test Error: 
 Accuracy: 28.9%, Avg loss:0.020679 

Test Error: 
 Accuracy: 28.0%, Avg loss:0.020730 

Test Error: 
 Accuracy: 28.3%, Avg loss:0.020692 

Test Error: 
 Accuracy: 28.0%, Avg loss:0.020788 

Test Error: 
 Accuracy: 28.1%, Avg loss:0.020773 

Test Error: 
 Accuracy: 28.2%, Avg loss:0.020844 

Test Error: 
 Accuracy: 28.5%, Avg loss:0.020806 

Test Error: 
 Accuracy: 28.6%, Avg loss:0.020774 

Test Error: 
 Accuracy: 28.1%, Avg loss:0.020887 

Test Error: 
 Accuracy: 28.6%, Avg loss:0.020863 

Test Error: 
 Accuracy: 28.3%, Avg loss:0.020852 

Test Error: 
 Accuracy: 28.5%, Avg loss:0.020994 

Test Error: 
 Accuracy: 28.3%, Avg loss:0.020825 

Test Error: 
 Accuracy: 28.7%, Avg loss:0.020978 

Test Error: 
 Accuracy: 28.5%, Avg loss:0.020977 

Test Error: 
 Accuracy: 28.4%, Avg loss:0.020963 

Test Error: 
 Accuracy: 28.2%, Avg loss:0.020877 

Test Error: 
 Accuracy: 28.3%, Avg loss:0.020906 

Test Error: 
 Accuracy: 28.7%, 

In [76]:
print(errrate)

0.29605263157894735


In [78]:
model.load_state_dict(torch.load('itemUser2.pth'))

<All keys matched successfully>

In [25]:
torch.save(model.state_dict(),'itemUser.pth')

In [79]:
model.eval()
with torch.no_grad():
    x_test = x_test.float()
    predO = model(x_test)
    pred = np.argmax(predO.cpu().numpy(),axis=1)
realpred = [ytestreal[i,j] for i,j in enumerate(pred)]

In [27]:
sampleSubmit = pd.read_csv('./submit_example.csv')

In [80]:
realpredict = pd.DataFrame({'user_id':sampleSubmit['user_id'],'product_id':np.array(realpred).reshape(-1)})

In [81]:
realpredict.to_csv('itemuserOF2.csv',index=False)